In [1]:
!pip install ibm-cos-sdk

     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 829kB 6.4MB/s 
     |████████████████████████████████| 225kB 25.4MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.6.3-py2.py3-none-any.whl size=72509 sha256=3263ea474d5107ee007556bb35e1fdd614b4895998cabe6e12921fb8b17a5118
  Stored in directory: /root/.cache/pip/wheels/23/c0/0b/6e7df83370c0fd314b4fe9aef6667e40178b31e7180dc69f26
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.6.3-py2.py3-none-any.whl size=500835 sha256=6109c1e801cea2428b767dd29e3ce4c39203b348a6c19926acec380007fce533
  Stored in directory: /root/.cache/pip/wheels/f1/29/05/ab91fed96d22f69b37c4331f33b77f0f05df0c382bf5f09167
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_sdk_s3transfer-2.6.3-py2.py3-none-any.whl size=88602 sha256=8a081fc39840631a16019fb5905dc11f25cbc74f7651b4139bfca2cda4f66108
  Stored in directory: /root/.cache/pip/wheels/89/7d/0a/2ff0215b826ef7d5a5292a62d0f57144976d7dc6

In [2]:
from __future__ import print_function
import json
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import ibm_boto3
import tensorflow as tf
from botocore.client import Config, ClientError

Using TensorFlow backend.


In [0]:
# Constants for IBM COS values
COS_ENDPOINT = "https://s3.us.cloud-object-storage.appdomain.cloud" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_API_KEY_ID = "zGB2jBX9oC6ZDu4ieC76TMhWTXpdR1HhbMHuBZ1_7ITA" # eg "W00YiRnLW4a3fTjMB-oiB-2ySfTrFBIQQWanc--P3byk"
COS_AUTH_ENDPOINT = "https://iam.cloud.ibm.com/identity/token"
COS_RESOURCE_CRN = "crn:v1:bluemix:public:cloud-object-storage:global:a/8d8f8f9cfc6fdd0b99e9f328b47ee768:855afe22-c6b1-4707-b1bf-23e1fcd64748::" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
BUCKET = "deeplearninglanguagemodelusingker-donotdelete-pr-cqy5sogxshcekv"
# Create resource
s3 = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint=COS_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [0]:
s3.Object(BUCKET, 'yelp_100_3.txt').download_file('yelp_100_3.txt')
s3.Object(BUCKET, 'char_indices.txt').download_file('char_indices.txt')
s3.Object(BUCKET, 'indices_char.txt').download_file('indices_char.txt')
s3.Object(BUCKET, 'transfer_weights').download_file('transfer_weights')

In [5]:
!ls -l

total 29896
-rw-r--r-- 1 root root      541 Jun  9 06:40 char_indices.txt
-rw-r--r-- 1 root root      663 Jun  9 06:40 indices_char.txt
drwxr-xr-x 1 root root     4096 May 29 18:19 sample_data
-rw-r--r-- 1 root root 30524436 Jun  9 06:40 transfer_weights
-rw-r--r-- 1 root root    71250 Jun  9 06:40 yelp_100_3.txt


In [6]:


path = 'yelp_100_3.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

char_indices = json.loads(open('char_indices.txt').read())
indices_char = json.loads(open('indices_char.txt').read())
chars = sorted(char_indices.keys())
print(indices_char)
#chars = sorted(list(set(text)))
print('total chars:', len(chars))
#char_indices = dict((c, i) for i, c in enumerate(chars))
#indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 256
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1




corpus length: 71250
{'0': '\n', '1': ' ', '2': '!', '3': '"', '4': '#', '5': '$', '6': '%', '7': '&', '8': "'", '9': '(', '10': ')', '11': '*', '12': '+', '13': ',', '14': '-', '15': '.', '16': '/', '17': '0', '18': '1', '19': '2', '20': '3', '21': '4', '22': '5', '23': '6', '24': '7', '25': '8', '26': '9', '27': ':', '28': ';', '29': '=', '30': '?', '31': '[', '32': ']', '33': 'a', '34': 'b', '35': 'c', '36': 'd', '37': 'e', '38': 'f', '39': 'g', '40': 'h', '41': 'i', '42': 'j', '43': 'k', '44': 'l', '45': 'm', '46': 'n', '47': 'o', '48': 'p', '49': 'q', '50': 'r', '51': 's', '52': 't', '53': 'u', '54': 'v', '55': 'w', '56': 'x', '57': 'y', '58': 'z', '59': '{', '60': '}'}
total chars: 61
nb sequences: 23665
Vectorization...


In [7]:

# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(LSTM(512, return_sequences=False))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = Adam(lr=0.002)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [8]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256, 1024)         4448256   
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense_1 (Dense)              (None, 61)                31293     
_________________________________________________________________
activation_1 (Activation)    (None, 61)                0         
Total params: 7,627,325
Trainable params: 7,627,325
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.load_weights("transfer_weights")
print('model loaded...')
def sample(preds, temperature=.6):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    x = np.zeros((1, maxlen, len(chars)))
    preds = model.predict(x, verbose=0)[0]
    
    model.fit(X, y, batch_size=128, epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)
    #start_index = char_indices["{"]

    for diversity in [0.2, 0.4, 0.6, 0.8]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            #print(next_index)
            print (indices_char)
            next_char = indices_char[str(next_index)]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
#model.save_weights("transfer_weights"

model loaded...

--------------------------------------------------
Iteration 1
Epoch 1/1
23665/23665 [==============================] - 330s 14ms/step - loss: 1.4069

----- diversity: 0.2
----- Generating with seed: "was cold again. one of them would rather eat glass then eat at this place so refused to order a cold meal, but since my other friend was paying she said fuck it i'll go.

well she ordered toast and some sausage as sides. and my other friend ordered the mis"
was cold again. one of them would rather eat glass then eat at this place so refused to order a cold meal, but since my other friend was paying she said fuck it i'll go.

well she ordered toast and some sausage as sides. and my other friend ordered the mis{'0': '\n', '1': ' ', '2': '!', '3': '"', '4': '#', '5': '$', '6': '%', '7': '&', '8': "'", '9': '(', '10': ')', '11': '*', '12': '+', '13': ',', '14': '-', '15': '.', '16': '/', '17': '0', '18': '1', '19': '2', '20': '3', '21': '4', '22': '5', '23': '6', '24': '7', '

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
t{'0': '\n', '1': ' ', '2': '!', '3': '"', '4': '#', '5': '$', '6': '%', '7': '&', '8': "'", '9': '(', '10': ')', '11': '*', '12': '+', '13': ',', '14': '-', '15': '.', '16': '/', '17': '0', '18': '1', '19': '2', '20': '3', '21': '4', '22': '5', '23': '6', '24': '7', '25': '8', '26': '9', '27': ':', '28': ';', '29': '=', '30': '?', '31': '[', '32': ']', '33': 'a', '34': 'b', '35': 'c', '36': 'd', '37': 'e', '38': 'f', '39': 'g', '40': 'h', '41': 'i', '42': 'j', '43': 'k', '44': 'l', '45': 'm', '46': 'n', '47': 'o', '48': 'p', '49': 'q', '50': 'r', '51': 's', '52': 't', '53': 'u', '54': 'v', '55': 'w', '56': 'x', '57': 'y', '58': 'z', '59': '{', '60': '}'}
 {'0': '\n', '1': ' ', '2': '!', '3': '"', '4': '#', '5': '$', '6': '%', '7': '&', '8': "'", '9': '(', '10': ')', '11': '*', '12': '+', '13': ',', '14': '-', '15': '.', '16': '/', '17': '0', '18': '1', '19': '2', '20': '3', '21': '4', '22': '5', '23': '6', '24': '7', '25': '8', '26': '9', '27': ':',